In [ ]:
# Data Loading, Storage, and File Formats: Chapter 6

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt

## Reading and Writing Data in Text Format

In [3]:
# Reading a small comma-separated (CSV) text file
path = 'C:/Users/aalaei/OneDrive - Southern Cross University/MyLapTop/SCU/MIT_Big_Data_Analysis_and_Tools/Designed Unit-S2-2020/Workshops/Examples_Book_Python for data analysis/'
fname1=path+'ex1.csv'
df = pd.read_csv(fname1)
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# use read_table and specified the delimiter
pd.read_table(fname1, sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# Reading a comma-separated (CSV) file having no column names
# You can allow pandas to assign default column names
fname2=path+'ex2.csv'
pd.read_csv(fname2, header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# you can specify names of the columns yourself
fname2=path+'ex2.csv'
pd.read_csv(fname2, names=['aa', 'bb', 'cc', 'dd', 'message'])

,aa,bb,cc,dd,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
# the message column to be the index of the returned DataFrame.
names1 = ['a', 'b', 'c', 'd', 'message']
pd.read_csv(fname2, names=names1, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [7]:
# you want to form a hierarchical index from multiple columns, pass a
# list of column numbers or names
fname3=path+'csv_mindex.csv'
parsed = pd.read_csv(fname3,index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [8]:
fname4=path+'ex3.txt'
list(open(fname4))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [9]:
# Missing data is usually either not present (empty string) or marked by
# some sentinel value. By default, pandas uses a set of commonly occurring sentinels,
# such as NA and NULL
fname5=path+'ex5.csv'
result = pd.read_table(fname5, sep=',')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [16]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [18]:
# The na_values option can take either a list or set of strings to 
# consider missing values
result = pd.read_csv(fname5, na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [15]:
# Different NA sentinels can be specified for each column in a dict:
sentinels = {'message': ['foo', 'NA'], 'something': ['three',]}
pd.read_csv(fname5, na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,NaN,9,10,11.0,12,NaN


# Reading Text Files in Pieces
When processing very large files or figuring out the right set of arguments to correctly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file. 

In [20]:
# we make the pandas display settings more compact showing only 10 rows
pd.options.display.max_rows = 10

In [16]:
fname6=path+'ex6.csv'
result = pd.read_csv(fname6)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [17]:
# If you want to only read a small number of rows (avoiding reading the entire file),
# specify that with nrows
df=pd.read_csv(fname6, nrows=5)
df

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [18]:
# To read a file in pieces, specify a chunksize as a number of rows
chunker = pd.read_csv(fname6, chunksize=1000)
chunker

In [28]:
# The TextParser object returned by read_csv allows you to iterate over the parts of
#the file according to the chunksize. 
#For example, we can iterate over ex6.csv, aggregating the value counts in the 'key' column
chunker = pd.read_csv(fname6, chunksize=1000)

tot = pd.Series([], dtype="float")
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [35]:
print(tot[:10], sum(tot))

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64 10000.0


### Writing Data to Text Format

In [20]:
mydata = pd.read_csv(fname5)
mydata

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
# Using DataFrame’s to_csv method, we can write the data out to a csv file
fnameo=path+'out.csv'
mydata.to_csv(fnameo)
#!type fnameo

In [23]:
# Other delimiters can be used
# writing to sys.stdout so it prints the text result to the console
import sys
mydata.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [27]:
# Missing values appear as empty strings in the output. 
# You might want to denote them by some other sentinel value:
mydata.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [28]:
# With no other options specified, both the row and column labels are written. 
# Both of these can be disabled:
mydata.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [29]:
# You can write only a subset of the columns, and in an order of your choice!
mydata.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [36]:
# Series also has a to_csv method:
fname7=path+'tseries.csv'
dates = pd.date_range('1/1/2000', periods=35)
ts = pd.Series(np.arange(35), index=dates)
ts.to_csv(fname7)
ts

2000-01-01     0
2000-01-02     1
2000-01-03     2
2000-01-04     3
2000-01-05     4
2000-01-06     5
2000-01-07     6
2000-01-08     7
2000-01-09     8
2000-01-10     9
2000-01-11    10
2000-01-12    11
2000-01-13    12
2000-01-14    13
2000-01-15    14
2000-01-16    15
2000-01-17    16
2000-01-18    17
2000-01-19    18
2000-01-20    19
2000-01-21    20
2000-01-22    21
2000-01-23    22
2000-01-24    23
2000-01-25    24
2000-01-26    25
2000-01-27    26
2000-01-28    27
2000-01-29    28
2000-01-30    29
2000-01-31    30
2000-02-01    31
2000-02-02    32
2000-02-03    33
2000-02-04    34
Freq: D, dtype: int32

### Working with Delimited Formats

In [37]:
# "a","b","c"
# "1","2","3"
# "1","2","3"
# For any file with a single-character delimiter, you can use Python’s built-in csv module.
# To use it, pass any open file or file-like object to csv.reader

import csv
fname8=path+'ex7.csv'
f = open(fname8)

reader = csv.reader(f)

In [38]:
# Iterating through the reader like a file yields tuples of values 
# with any quote characters removed
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [40]:
# From there, you can do the wrangling necessary to put the data in the form
# that you need it. To do so, first, read the file into a list of lines:
with open(fname8) as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [42]:
# Then, split the lines into the header line and the data lines:
header, values = lines[0], lines[1:]

In [43]:
# Then we can create a dictionary of data columns using a dictionary comprehension
# and the expression zip(*values), which transposes rows to columns
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
reader = csv.reader(f, dialect=my_dialect)

reader = csv.reader(f, delimiter='|')

with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### JSON Data

In [44]:
# JSON (short for JavaScript Object Notation) has become one of the standard formats
# for sending data by HTTP request between web browsers and other applications. 
# It is a much more free-form data format than a tabular text form like CSV.
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [45]:
# To convert a JSON string to Python form, use json.loads:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [3]:
# json.dumps converts a Python object back to JSON:
asjson = json.dumps(result)

In [47]:
# How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis
# Conveniently, you can pass a list of dicts the DataFrame constructor and select a subset of the data fields
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])
siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


In [8]:
# The pandas.read_json can automatically convert JSON datasets in specific arrangements
# into a Series or DataFrame
# The default options for pandas.read_json assume that each object in the JSON array
# is a row in the table:
fname9=path+'example.json'
data = pd.read_json(fname9)
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [9]:
# If you need to export data from pandas to JSON, one way is to use the to_json methods on Series and DataFrame
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping

conda install lxml
pip install beautifulsoup4 html5lib

In [48]:
# Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats.
# pandas has a built-in function, read_html, which uses libraries like lxml and Beautiful
# Soup to automatically parse tables out of HTML files as DataFrame objects.
fname10=path+'fdic_failed_bank_list.html'
tables = pd.read_html(fname10)
print(len(tables))
failures = tables[0]
failures.head()

1


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [49]:
# we can proceed to do some data cleaning and analysis, like computing the number of bank failures by year
update_timestamps = pd.to_datetime(failures['Updated Date'])
update_timestamps.dt.year.value_counts()

2012    227
2014    109
2016     91
2015     66
2013     36
2008      6
2005      5
2004      2
2003      2
2010      1
2006      1
2002      1
Name: Updated Date, dtype: int64

#### Parsing XML with lxml.objectify

In [15]:
# XML (eXtensible Markup Language) is another common structured data format supporting hierarchical,
# nested data with metadata.
    <INDICATOR>
        <INDICATOR_SEQ>373889</INDICATOR_SEQ>
        <PARENT_SEQ></PARENT_SEQ>
        <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
        <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
        <DESCRIPTION>Percent of the time that escalators are operational
        systemwide. The availability rate is based on physical observations performed
        the morning of regular business days only. This is a new indicator the agency
        began reporting in 2009.</DESCRIPTION>
        <PERIOD_YEAR>2011</PERIOD_YEAR>
        <PERIOD_MONTH>12</PERIOD_MONTH>
        <CATEGORY>Service Indicators</CATEGORY>
        <FREQUENCY>M</FREQUENCY>
        <DESIRED_CHANGE>U</DESIRED_CHANGE>
        <INDICATOR_UNIT>%</INDICATOR_UNIT>
        <DECIMAL_PLACES>1</DECIMAL_PLACES>
        <YTD_TARGET>97.00</YTD_TARGET>
        <YTD_ACTUAL></YTD_ACTUAL>
        <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
        <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
    </INDICATOR>

SyntaxError: invalid syntax (<ipython-input-15-f6eda7711f2e>, line 1)

In [50]:
# Using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot:

from lxml import objectify

fname12='C:/Users/aalaei/OneDrive - Southern Cross University/MyLapTop/SCU/MIT_Big_Data_Analysis_and_Tools/Designed Unit-S2-2020/Workshops/Datasets_Book_Python for data analysis/mta_perf/Performance_MNR.xml'

parsed = objectify.parse(open(fname12))
root = parsed.getroot()

In [51]:
# For each record, we can populate a dict of tag names (like YTD_ACTUAL) to data values 

data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [26]:
# Lastly, convert this list of dicts into a DataFrame:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


In [52]:
# XML data can get much more complicated than this example. Each tag can have
# metadata, too. Consider an HTML link tag, which is also valid XML:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [53]:
# You can now access any of the fields (like href) in the tag or the link text:
root
print(root.get('href'))
print(root.text)

http://www.google.com
Google


## Binary Data Formats

In [54]:
# One of the easiest ways to store data (also known as serialization) efficiently in binary
# format is using Python’s built-in pickle serialization. pandas objects all have a
# to_pickle method that writes the data to disk in pickle format

frame = pd.read_csv(fname1)
print(frame)
fname13=path+'frame_pickle'
frame.to_pickle(fname13)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [55]:
# You can read any “pickled” object stored in a file by using the built-in pickle directly,
# or even more conveniently using pandas.read_pickle:
pd.read_pickle(fname13)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo



### Using HDF5 Format

In [56]:
# HDF: hierarchical data format.
# HDF5 is a well-regarded file format intended for storing large quantities of scientific array data.
# The HDFStore class works like a dict and handles the low-level details:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [57]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [59]:
# HDFStore supports two storage schemas, 'fixed' and 'table'. The latter is generally
# slower, but it supports query operations using a special syntax
# The put is an explicit version of the store['obj2'] = frame method but allows us to
# set other options like the storage format.
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [60]:
# The pandas.read_hdf function gives you a shortcut to these tools:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [41]:
import os
os.remove('mydata.h5')

### Reading Microsoft Excel Files

In [62]:
# Pandas supports reading tabular data stored in Excel 2003 (and higher) files
# using either the ExcelFile class or pandas.read_excel function.
fname14=path+'ex1.xlsx'
xlsx = pd.ExcelFile(fname14)

In [63]:
# Data stored in a sheet can then be read into DataFrame with parse:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [44]:
# If you are reading multiple sheets in a file, then it is faster to create the ExcelFile,
# you can also simply pass the filename to pandas.read_excel
frame = pd.read_excel(fname14, 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [45]:
# To write pandas data to Excel format, you must first create an ExcelWriter, then
# write data to it using pandas objects’ to_excel method
fname15=path+'ex2.xlsx'
writer = pd.ExcelWriter(fname15)
frame.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
#You can also pass a file path to to_excel and avoid the ExcelWriter
frame.to_excel(fname15)

## Interacting with Web APIs

In [64]:
# Many websites have public APIs providing data feeds via JSON or some other format.
# There are a number of ways to access these APIs from Python
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [65]:
# The Response object’s json method will return a dictionary containing JSON parsed
# into native Python objects
data = resp.json()
data[0]['title']
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/35301',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/35301/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/35301/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/35301/events',
  'html_url': 'https://github.com/pandas-dev/pandas/issues/35301',
  'id': 657717188,
  'node_id': 'MDU6SXNzdWU2NTc3MTcxODg=',
  'number': 35301,
  'title': 'BUG: xs not working with slice',
  'user': {'login': 'wiso',
   'id': 143389,
   'node_id': 'MDQ6VXNlcjE0MzM4OQ==',
   'avatar_url': 'https://avatars3.githubusercontent.com/u/143389?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/wiso',
   'html_url': 'https://github.com/wiso',
   'followers_url': 'https://api.github.com/users/wiso/followers',
   'following_url': 'https://api.github.com/users/wiso/following{

In [66]:
# Each element in data is a dictionary containing all of the data found on a GitHub
# issue page (except for the comments). We can pass data directly to DataFrame and
# extract fields of interest
issues = pd.DataFrame(data, columns=['id', 'user', 'number', 'title',
                                     'labels', 'state'])
issues

,id,user,number,title,labels,state
0,657717188,"{'login': 'wiso', 'id': 143389, 'node_id': 'MD...",35301,BUG: xs not working with slice,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
1,657696528,"{'login': 'erfannariman', 'id': 34067903, 'nod...",35300,ENH: add percentage threshold to dropna,[],open
2,657653272,"{'login': 'erfannariman', 'id': 34067903, 'nod...",35299,ENH: add percentage threshold to DataFrame.dropna,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
3,657624935,"{'login': 'TomAugspurger', 'id': 1312546, 'nod...",35298,TST: Remove deprecated use of apply_index,[],open
4,657622552,"{'login': 'aleisun', 'id': 40581023, 'node_id'...",35297,BUG: AmbiguousTimeError when using date_range ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
5,657610610,"{'login': 'sm-Fifteen', 'id': 516999, 'node_id...",35296,DOC: Reccomended use of read_csv's date_parser...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
6,657597800,"{'login': 'TomAugspurger', 'id': 1312546, 'nod...",35294,Various 32-bit failures at MacPython/pandas-wh...,"[{'id': 563047854, 'node_id': 'MDU6TGFiZWw1NjM...",open
7,657546310,"{'login': 'rwijtvliet', 'id': 4106013, 'node_i...",35293,BUG: upsampling returns less rows than expected,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
8,657531275,"{'login': 'Dr-Irv', 'id': 15113894, 'node_id':...",35292,ENH: Add orient=tight format for dictionaries,[],open
9,657443705,"{'login': 'TomAugspurger', 'id': 1312546, 'nod...",35288,TestUltraJSONTests.test_dumps_ints_larger_than...,"[{'id': 563047854, 'node_id': 'MDU6TGFiZWw1NjM...",open


## Interacting with Databases

In [68]:
# Loading data from SQL into a DataFrame is fairly straightforward, and pandas has
# some functions to simplify the process. As an example, a SQLite database is created
# using Python’s built-in sqlite3 driver
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata1.sqlite')
con.execute(query)
con.commit()

In [69]:
# Insert a few rows of data:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [70]:
# Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list
# of tuples when selecting data from a table
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [58]:
# You can pass the list of tuples to the DataFrame constructor, but you also need the
# column names, contained in the cursor’s description attribute
print(cursor.description)
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

(('a', None, None, None, None, None, None), ('b', None, None, None, None, None, None), ('c', None, None, None, None, None, None), ('d', None, None, None, None, None, None))


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [71]:
# The SQLAlchemy project is a popular Python SQL toolkit that abstracts
# away many of the common differences between SQL databases. pandas has a
# read_sql function that enables you to read data easily from a general SQLAlchemy
# connection. Here, we’ll connect to the same SQLite database with SQLAlchemy and
# read data from the table created before
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata1.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## Conclusion